In [1]:
import pandas as pd
import numpy as np
from models.MKNN import ModifiedKNN
import neattext.functions as nfx
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
df = pd.read_csv('Twitter_fresh/twitter_crawling.csv',encoding='latin1', usecols=['date','text'])
df.head()

# Preprocessing

In [ ]:
def casefolding(Text):
    Text = Text.lower()
    return Text

In [ ]:
df['text'] = df['text'].apply(casefolding)
df.head()

In [ ]:
def punc_clean(Text):
        Text = nfx.remove_urls(Text)
        Text = nfx.remove_punctuations(Text)
        Text = nfx.remove_emojis(Text)
        Text = nfx.remove_special_characters(Text)
        Text = nfx.remove_numbers(Text)
        return Text

In [ ]:
df['text'] = df['text'].apply(punc_clean)
df.head()

In [ ]:
def word_tokenize_wrapper(Text):
        return word_tokenize(Text)

In [ ]:
df['text'] = df['text'].apply(word_tokenize_wrapper)
df.head()

In [ ]:
def word_norm(tweets):
    word_dict = pd.read_csv('data/indonesia_slangWords.csv')
    norm_word_dict = {}
    for index, row in word_dict.iterrows():
        if row[0] not in norm_word_dict:
            norm_word_dict[row[0]] = row[1]
    return [norm_word_dict[term] if term in norm_word_dict else term for term in tweets]

In [ ]:
df['text'] = df['text'].apply(word_norm)
df.head()

In [ ]:
def remove_stopword(Text):
    stopW = stopwords.words('indonesian', 'english')
    sw = pd.read_csv('data/stopwordbahasa.csv')
    stopW.extend(sw)
    remove_sw = ' '.join(Text)
    clean_sw = [word for word in remove_sw.split() if word.lower() not in stopW]
    return clean_sw

In [ ]:
df['text'] = df['text'].apply(remove_stopword)
df.head()

In [ ]:
def indo_stem(Text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    result = []
    for w in Text:
        result.append(stemmer.stem(w))
        result.append(" ")
    return " ".join(result)

In [ ]:
df['text'] = df['text'].apply(indo_stem)
df.head()

# Sentiment

In [2]:
from sklearn.model_selection import KFold
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json

In [3]:
def Sentiment_analysis():
    df = pd.read_csv('Twitter_fresh\Clean Data\Twitter_clean_1000.csv', encoding='utf-8')

    # LexiconVader dengan custom Lexicon(bahasa indonesia)
    sia1A, sia1B = SentimentIntensityAnalyzer(), SentimentIntensityAnalyzer()
    senti = SentimentIntensityAnalyzer()
    # Hapus Default lexicon VADER
    sia1A.lexicon.clear()
    sia1B.lexicon.clear()
    senti.lexicon.clear()

    # Read custom Lexicon Bahasa Indonesia
    data1A = open('data/lexicon_sentimen_negatif.txt', 'r').read()
    data1B = open('data/lexicon_sentimen_positif.txt', 'r').read()
    data_senti = open('data/sentiwords_id.txt', 'r').read()
    
    # convert lexicon to dictonary
    insetNeg = json.loads(data1A)
    insetPos = json.loads(data1B)
    sa = json.loads(data_senti)

    # update lexicon vader with custom lexicon (b.indo)
    sia1A.lexicon.update(insetNeg)
    sia1B.lexicon.update(insetPos)
    senti.lexicon.update(sa)

    # method untuk cek apa sentimen pos,neg,neu
    def is_positive_inset(Text: str) -> bool:
        return sia1A.polarity_scores(Text)["compound"] + sia1B.polarity_scores(Text)["compound"] >= 0.05
    
    tweets = df['text'].to_list()

    with open('output/Sentiment-result.txt', 'w+') as f:
        for tweet in tweets:
            label = "Positive" if is_positive_inset(tweet) else "Negative"
            f.write(str(label + "\n"))

    sen = pd.read_csv('output/Sentiment-result.txt', names=['Sentiment'])
    df = df.join(sen)

    ## Save clean Dataset
    #df.to_csv('CleanText_Sentiment.csv', index=False)
    return df

In [4]:
sentiment_result = Sentiment_analysis()

In [5]:
sentiment_result['Sentiment'].value_counts()

Negative    791
Positive    209
Name: Sentiment, dtype: int64

In [ ]:
sentiment_result.to_csv('output/Sentiment_result.csv', index=False)

# Models Algorithm

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
from sklearn.metrics import f1_score, recall_score
from heapq import nsmallest as nMin
from tqdm import tqdm

In [123]:
k_value = 9 # 1 - 25
X = sentiment_result['text'].values
y = sentiment_result['Sentiment'].values
fold_i = 1
fold_n = 10 # 3 5 7 10
sum_accuracy = 0
kfold = KFold(fold_n, shuffle=True, random_state=42)
enc = LabelEncoder()
fol = []
acc, rc, pr, f1 = [], [], [], []

for train_index, test_index in tqdm(kfold.split(X)):
    fol.append(fold_i)
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    
    svf = open('output/ResultX.txt', 'w')
    sv_text = '\n'.join(str(item) for item in X_test).replace("   "," ")
    svf.write(sv_text)
    svY = open ('output/y_train.txt', 'w')
    svY.write('\n'.join(str(item) for item in y_train))

    #TFIDF
    tf = TfidfVectorizer(decode_error="replace")
    X_train = tf.fit_transform(X_train)
    X_test = tf.transform(X_test)
    
    y_train = enc.fit_transform(y_train)
    y_test = enc.transform(y_test)

    # Algorithm
    clf = ModifiedKNN(k_value)
    clf.fit(X_train, y_train)
    pred, jarak = clf.predict(X_test)
    neigbor_index = clf.get_neigbors(X_test)

    # Confusion Matrix
    #tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
    accuracy = accuracy_score(y_test, pred)*100
    precision = precision_score(y_test, pred)*100
    recall = recall_score(y_test, pred)*100
    f1_scores = f1_score(y_test, pred)*100
    #accuracy = (tp + tn) / (tp + fp + tn + fn)*100
    #precision = (tp) / (tp + fp)*100
    #recall = (tp) / (tp + fn)*100
    #f1_scores = (2 * precision * recall) / (precision + recall)
    #plot_conf_metrics(y_test, pred)

    sum_accuracy += accuracy
    pred = enc.inverse_transform(pred)

    fold_i += 1
    acc.append(accuracy)
    pr.append(precision)
    rc.append(recall)
    f1.append(f1_scores)

4it [00:03,  1.16it/s]c:\Users\Alexander Adam\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
10it [00:08,  1.14it/s]


In [124]:
with open("output/MKNN_prediction.txt", "w") as f:
    mknn_predited_label ='\n'.join(str(item) for item in pred)
    f.write(mknn_predited_label)
with open('output/jarak_ttg.txt', 'w') as g:
    jarak = [nMin(k_value,map(float,i)) for i in jarak]
    mknn_distance = '\n'.join(str(ls) for ls in jarak)
    g.write(mknn_distance)
with open('output/index_ttg.txt', 'w') as j:
    j.write('\n'.join(str(a) for a in neigbor_index))

In [125]:
knn_pred = pd.read_csv('output/MKNN_prediction.txt', names=['Sentiment'])
jarak_pred = pd.read_csv('output/jarak_ttg.txt', names=['Distance'], sep='\t')
text_test = pd.read_csv('output/ResultX.txt', names=['text'])
index_pred = pd.read_csv('output/index_ttg.txt', names=['Neigbor'])
text_test = text_test.join(knn_pred)
text_test = text_test.join(jarak_pred)
text_test = text_test.join(index_pred)
#text_test['Sentiment'] = text_test['Sentiment'].apply(lambda x: 'Positive' if x == 1 else 'Negative')
text_test = text_test.dropna()

In [126]:
text_test.head()

,text,Sentiment,Distance,Neigbor
0,money heist suka curi,Negative,"[1.1165595996858388, 1.13578387669024, 1.16036...",[733 295 701 699 676 474 836 5 65]
1,sebel banget deh nonton film film bagus netfli...,Negative,"[0.9862296931918912, 1.1235266221003684, 1.154...",[ 8 477 796 59 127 694 741 339 873]
2,mimpi tokyo money heist,Negative,"[0.8488482062392781, 0.9300298921025985, 0.930...",[816 496 278 643 465 878 509 407 256]
3,season nya money heist nonton nafas anjirrr en...,Negative,"[1.0747805724581887, 1.1554921358307253, 1.229...",[319 281 833 456 424 264 556 468 101]
4,pikir deh agency tema nya crime agency cari vo...,Negative,"[1.8250120749944284e-08, 1.2291833293009768, 1...",[ 17 271 330 16 21 273 154 492 217]


In [127]:
new_frame = pd.DataFrame(X_test)
new_frame = new_frame.join(knn_pred)

avg_acc = sum_accuracy/fold_n
maxs = max(acc)
mins = min(acc)
res_df = pd.DataFrame({'iterasi':fol, 'Accuracy': acc, 'Precison':pr, 'Recall':rc, 'f1 score':f1})
print("Avearge accuracy : ", str("%.4f" % avg_acc)+'%')
print("Max Score : ",str(maxs),"in Fold : ", str(acc.index(maxs)+1))
print("Min Score : ",str(mins), "in Fold : ", str(acc.index(mins)+1))

Avearge accuracy :  78.9000%
Max Score :  85.0 in Fold :  9
Min Score :  75.0 in Fold :  3


In [128]:
res_df.head(11)

,iterasi,Accuracy,Precison,Recall,f1 score
0,1,77.0,33.333333,4.545455,8.000000
1,2,76.0,100.000000,4.000000,7.692308
2,3,75.0,66.666667,7.692308,13.793103
3,4,77.0,25.000000,4.761905,8.000000
4,5,82.0,0.000000,0.000000,0.000000
5,6,81.0,0.000000,0.000000,0.000000
6,7,76.0,40.000000,8.695652,14.285714
7,8,78.0,100.000000,4.347826,8.333333
8,9,85.0,100.000000,11.764706,21.052632
9,10,82.0,33.333333,5.882353,10.000000


In [122]:
import matplotlib.pyplot as plot

In [ ]:
res_df[['Accuracy', 'Precison', 'Recall', 'f1 score']].plot.line(title="Akurasi tiap Fold")
plot.show(block=True)